In [1]:
import csv
from os import listdir
from os.path import isfile, join
import _pickle as pickle
import numpy as np

In [13]:
data_files_dir = "/Users/dohk/GoogleDrive/Bithumb_Data_Collector/cleanup"
# cleanup된 raw data의 위치

In [3]:
all_data_files = [f for f in listdir(data_files_dir) if isfile(join(data_files_dir, f))]
# 디렉토리에 존재하는 모든 파일의 이름을 리스트로 저장

In [4]:
all_data_files

['coin_QTUM_cleanup.csv',
 'coin_ETH_cleanup.csv',
 'Icon\r',
 'coin_ZEC_cleanup.csv',
 'coin_XRP_cleanup.csv',
 'coin_BTG_cleanup.csv',
 'coin_EOS_cleanup.csv',
 'coin_XMR_cleanup.csv',
 'coin_BCH_cleanup.csv',
 'coin_LTC_cleanup.csv',
 'coin_BTC_cleanup.csv',
 'coin_DASH_cleanup.csv',
 'coin_ETC_cleanup.csv']

In [7]:
trading_coins = ['BCH', 'BTC', 'DASH', 'ETC', 'ETH', 'LTC', 'XRP']
trading_files = [f for f in all_data_files for coin in trading_coins if coin in f]
trading_files
# 데이터셋으로 만들고자 하는 코인의 이름을 리스트로 저장

['coin_ETH_cleanup.csv',
 'coin_XRP_cleanup.csv',
 'coin_BCH_cleanup.csv',
 'coin_LTC_cleanup.csv',
 'coin_BTC_cleanup.csv',
 'coin_DASH_cleanup.csv',
 'coin_ETC_cleanup.csv']

unix timestamp: 1518050400000 ~ <br>
human readable time: 2018.02.08 9:30 ~ <br>
unix timestamp: 1519159200000 <br>
human readable time: 2018.02.21 5:40<br>

In [97]:
all_data_list = dict() #전체 데이터
for f in trading_files:
    all_data_list[f] = list()
    
start_ms_time = 1518050400000
end_ms_time = 1519159200000 #1519129800000 #1519159200000

for f in trading_files:  # 각 코인별로
    file = open(data_files_dir + '/' + f, 'r', encoding='utf-8') 
    rdr = csv.reader(file)  # read mode로 열고
    for line in rdr:  # 읽은 csv파일을 라인별로
        if start_ms_time <= int(line[0]) and end_ms_time >= int(line[0]):  # 뭘 검사하냐면, 시간을.
            all_data_list[f].append(line)  # 해당 코인의 key값에 대한 value로 해당 라인을 저장.
        else:
            pass  # starttime~endtime범위를 넘어간 데이터는 패스
    file.close()
    
for i in trading_files:
    print(i, ":", len(all_data_list[i]))

coin_ETH_cleanup.csv : 1849
coin_XRP_cleanup.csv : 1849
coin_BCH_cleanup.csv : 1849
coin_LTC_cleanup.csv : 1849
coin_BTC_cleanup.csv : 1849
coin_DASH_cleanup.csv : 1849
coin_ETC_cleanup.csv : 1849


In [86]:
all_data_list['coin_ETH_cleanup.csv'][0]

['1518050400000',
 '2018-02-08 09:40:00',
 '867000',
 '855000',
 '868000',
 '850000',
 '1614.3800521976',
 'normal']

In [91]:
all_data_list['coin_ETH_cleanup.csv'][-1]

['1519159200000',
 '2018-02-21 05:40:00',
 '1084000',
 '1082000',
 '1085000',
 '1082000',
 '118.37528202765',
 'normal']

In [98]:
all_data_list = dict() #전체 데이터
for f in trading_files:
    all_data_list[f] = list()
    
start_ms_time = 1501850400000
end_ms_time = 1519159200000 #1519129800000 #1519159200000

for f in trading_files:  # 각 코인별로
    file = open(data_files_dir + '/' + f, 'r', encoding='utf-8') 
    rdr = csv.reader(file)  # read mode로 열고
    for line in rdr:  # 읽은 csv파일을 라인별로
        if start_ms_time <= int(line[0]) and end_ms_time >= int(line[0]):  # 뭘 검사하냐면, 시간을.
            all_data_list[f].append(line)  # 해당 코인의 key값에 대한 value로 해당 라인을 저장.
        else:
            pass  # starttime~endtime범위를 넘어간 데이터는 패스
    file.close()
    
for i in trading_files:
    print(i, ":", len(all_data_list[i]))

coin_ETH_cleanup.csv : 28849
coin_XRP_cleanup.csv : 28849
coin_BCH_cleanup.csv : 28849
coin_LTC_cleanup.csv : 28849
coin_BTC_cleanup.csv : 28849
coin_DASH_cleanup.csv : 28849
coin_ETC_cleanup.csv : 28849


In [111]:
trv = list() #train value : [[v_t, v_t_hi, v_t_low], ...], shape : (27000, 3, 7, 50)
btv = list() #backtest value : [v_t, v_t_hi, v_t_low], ...], shape : (1800, 3, 7, 50)

trv_file = open(data_files_dir + '/dataset/trv', 'wb')
btv_file = open(data_files_dir + '/dataset/btv', 'wb')

In [117]:
data_iterator = dict()
for file in trading_files:
    data_iterator[file] = iter(all_data_list[file])

In [118]:
v_t = list() #shape : (7, 50)
v_t_hi = list() #(7, 50)
v_t_low = list() #(7, 50)
v_t_f = list() #shape : (50)
v_t_hi_f = list() #(50)
v_t_low_f = list() #(50)

In [99]:
while True:
    try:
        for f in trading_files:
            if not v_t_f == list():
                v = next(data_iterator[f])
                if v == None:
                    break
                v_t_f = v_t_f[1:]
                v_t_hi_f = v_t_hi_f[1:]
                v_t_low_f = v_t_low_f[1:]
                v_t_f.append(v[3])
                v_t_hi_f.append(v[4])
                v_t_low_f.append(v[5])
            else:
                for i in range(50):
                    v = next(data_iterator[f])
                    v_t_f.append(v[3])
                    v_t_hi_f.append(v[4])
                    v_t_low_f.append(v[5])
            v_t.append(v_t_f)
            v_t_hi.append(v_t_hi_f)
            v_t_low.append(v_t_low_f)

        if len(trv) < 27000:
            trv.append([v_t, v_t_hi, v_t_low])
            v_t = list()
            v_t_hi = list()
            v_t_low = list()
        else:
            btv.append([v_t, v_t_hi, v_t_low])
            v_t = list()
            v_t_hi = list()
            v_t_low = list()
    except StopIteration:
        break

trv = np.array(trv, dtype=np.float64)
btv = np.array(btv, dtype=np.float64)

print("train dataset's shape :", trv.shape)
print("backtest dataset's shape :", btv.shape)

print("example : ", trv[0])

pickle.dump(trv, trv_file, protocol=4)
pickle.dump(btv, btv_file, protocol=4)

trv_file.close()
btv_file.close()

train dataset's shape : (27000, 3, 7, 50)
backtest dataset's shape : (1800, 3, 7, 50)
example :  [[[  2.52250000e+05   2.51200000e+05   2.51750000e+05 ...,   2.52550000e+05
     2.52300000e+05   2.52650000e+05]
  [  2.51200000e+05   2.51750000e+05   2.51950000e+05 ...,   2.52300000e+05
     2.52650000e+05   1.95000000e+02]
  [  2.51750000e+05   2.51950000e+05   2.52100000e+05 ...,   2.52650000e+05
     1.95000000e+02   8.60000000e+05]
  ..., 
  [  2.52100000e+05   2.52000000e+05   2.52200000e+05 ...,   8.60000000e+05
     4.82000000e+04   3.14500000e+06]
  [  2.52000000e+05   2.52200000e+05   2.52950000e+05 ...,   4.82000000e+04
     3.14500000e+06   2.11000000e+05]
  [  2.52200000e+05   2.52950000e+05   2.52500000e+05 ...,   3.14500000e+06
     2.11000000e+05   1.69650000e+04]]

 [[  2.52900000e+05   2.52300000e+05   2.51750000e+05 ...,   2.52800000e+05
     2.52850000e+05   2.52650000e+05]
  [  2.52300000e+05   2.51750000e+05   2.52850000e+05 ...,   2.52850000e+05
     2.52650000e+05

In [103]:
len(trv)

27000

In [106]:
print(trv[0])
print(len(trv[0])) # closing price, high price, low price

[[[  2.52250000e+05   2.51200000e+05   2.51750000e+05 ...,   2.52550000e+05
     2.52300000e+05   2.52650000e+05]
  [  2.51200000e+05   2.51750000e+05   2.51950000e+05 ...,   2.52300000e+05
     2.52650000e+05   1.95000000e+02]
  [  2.51750000e+05   2.51950000e+05   2.52100000e+05 ...,   2.52650000e+05
     1.95000000e+02   8.60000000e+05]
  ..., 
  [  2.52100000e+05   2.52000000e+05   2.52200000e+05 ...,   8.60000000e+05
     4.82000000e+04   3.14500000e+06]
  [  2.52000000e+05   2.52200000e+05   2.52950000e+05 ...,   4.82000000e+04
     3.14500000e+06   2.11000000e+05]
  [  2.52200000e+05   2.52950000e+05   2.52500000e+05 ...,   3.14500000e+06
     2.11000000e+05   1.69650000e+04]]

 [[  2.52900000e+05   2.52300000e+05   2.51750000e+05 ...,   2.52800000e+05
     2.52850000e+05   2.52650000e+05]
  [  2.52300000e+05   2.51750000e+05   2.52850000e+05 ...,   2.52850000e+05
     2.52650000e+05   1.95000000e+02]
  [  2.51750000e+05   2.52850000e+05   2.52400000e+05 ...,   2.52650000e+05
  

In [107]:
print(len(trv[0][0])) # 코인 개수
print(trv[0][0])

7
[[  2.52250000e+05   2.51200000e+05   2.51750000e+05   2.51950000e+05
    2.52100000e+05   2.52000000e+05   2.52200000e+05   2.52950000e+05
    2.52500000e+05   2.52600000e+05   2.52350000e+05   2.52000000e+05
    2.52050000e+05   2.51950000e+05   2.51400000e+05   2.50800000e+05
    2.50900000e+05   2.51400000e+05   2.51500000e+05   2.51600000e+05
    2.51850000e+05   2.52300000e+05   2.52050000e+05   2.51700000e+05
    2.51750000e+05   2.51900000e+05   2.52150000e+05   2.51900000e+05
    2.51850000e+05   2.51850000e+05   2.51600000e+05   2.51700000e+05
    2.51550000e+05   2.51400000e+05   2.51250000e+05   2.51350000e+05
    2.51500000e+05   2.51400000e+05   2.51650000e+05   2.51650000e+05
    2.51600000e+05   2.51700000e+05   2.51900000e+05   2.51700000e+05
    2.52300000e+05   2.52400000e+05   2.52050000e+05   2.52550000e+05
    2.52300000e+05   2.52650000e+05]
 [  2.51200000e+05   2.51750000e+05   2.51950000e+05   2.52100000e+05
    2.52000000e+05   2.52200000e+05   2.52950000e+0

In [108]:
print(len(trv[0][0][0])) # 논문 "n is the number of input periods before t"
print(trv[0][0][0])

50
[ 252250.  251200.  251750.  251950.  252100.  252000.  252200.  252950.
  252500.  252600.  252350.  252000.  252050.  251950.  251400.  250800.
  250900.  251400.  251500.  251600.  251850.  252300.  252050.  251700.
  251750.  251900.  252150.  251900.  251850.  251850.  251600.  251700.
  251550.  251400.  251250.  251350.  251500.  251400.  251650.  251650.
  251600.  251700.  251900.  251700.  252300.  252400.  252050.  252550.
  252300.  252650.]
